In [ ]:
''' 2 external files have been used
i. preprocessed csv (contains preprocessed text in column 'processed_text)
ii. corpus.txt. this can be generated from the preprocessed csv via the commented out code below'''

" 2 external files have been used \n1. preprocessed csv (contains preprocessed text in column 'processed_text)\n2. corpus.txt. this can be generated from the preprocessed csv via the commented out code below"

In [1]:

import gensim
import random
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from tqdm import tqdm
from collections import Counter


In [2]:
# only for use in colab


drive_path = './drive/MyDrive/Colab Notebooks'
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
file_path = f'{drive_path}/preprocessed_TOI_FarmersProtests.csv'
data = pd.read_csv(file_path)

data['processed_text'] = data['processed_text'].astype(str)

sentences = data['processed_text'].apply(lambda x: x.split()).tolist()
# sentences now has sentence-level corpus, this will only ever be used for training


In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('./drive/MyDrive/Colab Notebooks/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
#use this to construct the file 'corpus.txt', which is simply a string of the entire corpus, each words seperated by a whitespace

# def csv_column_to_txt(csv_file_path, txt_file_path, column_name):


#     with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
#             csv_reader = csv.DictReader(csv_file)
#             count = 0


#             with open(txt_file_path, 'w') as txt_file:

#                 for row in csv_reader:

#                     if count % 100 == 0:
#                         print("Done with ", count, " rows")
#                     count += 1

#                     text = row[column_name]
#                     txt_file.write(text)

#     print(f"Words from column '{column_name}' in '{csv_file_path}' have been added to {txt_file_path}.")
#     return None

# csv_file_path = ".\preprocessed_TOI_FarmersProtests.csv"
# txt_file_path = ".\corpus.txt"
# column_name = "processed_text"

# csv_column_to_txt(csv_file_path, txt_file_path, column_name)





In [5]:
# file_path = './corpus.txt'
file_path = './drive/MyDrive/Colab Notebooks/corpus.txt'

with open(file_path, 'r') as file:
    text = file.read()

text = text.split()

print(len(text))
# for reference, there are a total of 7,19,468 words

719468


In [28]:
unique_words = set([word for word in text]) #the number of unique words reduces down to just 41131
unique_words_list = [word for word in unique_words]

unique_word_vectors = [model[word] for word in unique_words if word in model]

unique_word_vectors_array = np.array(unique_word_vectors)

print(f"Total {len(unique_words_list)} unique words, {len(unique_word_vectors_array)} in-vocabulary words")

Total 41131 unique words, 16151 in-vocabulary words


In [29]:
# Calculate mean and standard deviation
mean = np.mean(unique_word_vectors_array)
std = np.std(unique_word_vectors_array)

print(f"Mean: {mean}, Standard Deviation: {std}")

Mean: -0.004620064981281757, Standard Deviation: 0.17973092198371887


In [7]:
# getting initial embeddings (pre-trained google news + randomized OOVs)
def get_word_vector(word, model, mean, std, embedding_dim=300):
    if word in model:
        return model[word]
    else:
        return np.random.normal(mean, std, embedding_dim)

In [48]:
# get top-n words and embeddings
word_counts = Counter(text)

top_n = 1000
top_words = [word for word, _ in word_counts.most_common(top_n)]

top_embeddings = [get_word_vector(word, model, mean, std, 300) for word in top_words]

top_words.sort()
print(top_words)


['aap', 'able', 'accept', 'accident', 'accord', 'account', 'accuse', 'acre', 'act', 'action', 'activist', 'activity', 'add', 'addition', 'additional', 'address', 'administration', 'adopt', 'affect', 'afternoon', 'agency', 'agitate', 'agitation', 'ago', 'agra', 'agree', 'agreement', 'agri', 'agricultural', 'agriculture', 'ahead', 'aim', 'air', 'akali', 'alert', 'allege', 'allegedly', 'allow', 'amarinder', 'amendment', 'amid', 'andhra', 'animal', 'announce', 'announcement', 'antifarmer', 'apart', 'apmc', 'appeal', 'appear', 'application', 'approach', 'april', 'area', 'arrangement', 'arrest', 'arrive', 'ashok', 'ask', 'assembly', 'assess', 'assistance', 'association', 'assurance', 'assure', 'attack', 'attempt', 'attend', 'auction', 'august', 'authority', 'avail', 'available', 'average', 'avoid', 'award', 'away', 'b', 'badal', 'bag', 'ban', 'banana', 'bandh', 'bank', 'barricade', 'base', 'basis', 'begin', 'belong', 'bench', 'beneficiary', 'benefit', 'bengal', 'bharat', 'bharatiya', 'big', 

In [49]:
# plot PCA

def plot_with_plotly_PCA(top_embeddings, text, sample_size=top_n, save_as=None): #pass a string argument for filename (save_as) to save the plot
    pca = PCA(n_components = 2)
    reduced_embeddings = pca.fit_transform(top_embeddings)
    print('PCA reduction done')

    indices = random.sample(range(len(text)), sample_size)
    sampled_embeddings = reduced_embeddings[indices]
    sampled_text = [text[i] for i in indices]

    df = pd.DataFrame({
        'PCA1': sampled_embeddings[:, 0],
        'PCA2': sampled_embeddings[:, 1],
        'text': sampled_text
    })

    fig = px.scatter(df, x='PCA1', y='PCA2', text='text', title='PCA of Text Embeddings')
    fig.update_traces(textposition='top center')
    fig.show()

    if save_as != None:
      fig.write_html(f"{drive_path}/{save_as}.html") #saves the plots in the used gdrive if using google collab. If not, specify a local path



plot_with_plotly_PCA(top_embeddings, top_words, top_n, 'pre-training-PCA') #having the last param to be top_n skips sampling and instead plots everything


PCA reduction done


In [50]:
# plot tSNE

def plot_with_plotly_tsne(top_embeddings, text, sample_size=top_n, save_as=None):

    top_embeddings_array = np.array(top_embeddings)
    tsne = TSNE(perplexity=2, n_components=2, init='pca', n_iter=1000, random_state=23, n_jobs=-1, method='barnes_hut', verbose=1)
    reduced_embeddings_tsne = tsne.fit_transform(np.array (top_embeddings_array))


    print('tSNE reduction done')

    indices = random.sample(range(len(text)), sample_size)
    sampled_embeddings = reduced_embeddings_tsne[indices]
    sampled_text = [text[i] for i in indices]

    df = pd.DataFrame({
        'TSNE1': sampled_embeddings[:, 0],
        'TSNE2': sampled_embeddings[:, 1],
        'text': sampled_text
    })

    fig = px.scatter(df, x='TSNE1', y='TSNE2', text='text', title='t-SNE of Text Embeddings')
    fig.update_traces(textposition='top center')
    fig.show()

    if save_as != None:
          fig.write_html(f"{drive_path}/{save_as}.html") #saves the plots in the used gdrive if using google collab. If not, specify a local path


plot_with_plotly_tsne(top_embeddings, top_words, top_n, 'pre-training-tSNE')

[t-SNE] Computing 7 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.005s...
[t-SNE] Computed neighbors for 1000 samples in 0.027s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.342124
[t-SNE] KL divergence after 250 iterations with early exaggeration: 101.409286
[t-SNE] KL divergence after 1000 iterations: 2.036375
tSNE reduction done


In [39]:
#training

trained_model_cbow = Word2Vec(sentences, vector_size=300, window=5, min_count=2, workers=4, sg=0, epochs=10) # CBOW
trained_model_skipgram = Word2Vec(sentences, vector_size=300, window=5, min_count=2, workers=4, sg=1, epochs=10) # SKIPGRAM


In [51]:
# plotting trained embeddings CBOW

top_embeddings_trained_cbow = [trained_model_cbow.wv[word] for word in top_words]
plot_with_plotly_PCA(top_embeddings_trained_cbow, top_words, top_n, 'cbow-PCA')
plot_with_plotly_tsne(top_embeddings_trained_cbow, top_words, top_n, 'cbow-tSNE')

PCA reduction done


[t-SNE] Computing 7 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.002s...
[t-SNE] Computed neighbors for 1000 samples in 0.036s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.133317
[t-SNE] KL divergence after 250 iterations with early exaggeration: 84.840714
[t-SNE] KL divergence after 1000 iterations: 1.178156
tSNE reduction done


In [52]:
# plotting trained embeddings Skip-Gram

top_embeddings_trained_skipgram = [trained_model_skipgram.wv[word] for word in top_words]
plot_with_plotly_PCA(top_embeddings_trained_skipgram, top_words, top_n, 'skipgram-PCA')
plot_with_plotly_tsne(top_embeddings_trained_skipgram, top_words, top_n, 'skipgram-TSNE')

PCA reduction done


[t-SNE] Computing 7 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.034s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.505642
[t-SNE] KL divergence after 250 iterations with early exaggeration: 90.308647
[t-SNE] KL divergence after 1000 iterations: 1.487254
tSNE reduction done
